In [ ]:
# Agent with Memory

In [ ]:
## Setup model

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv() # Load environment variables from .env file

az_o3_model_client = AzureOpenAIChatCompletionClient(
    model = "o3-mini",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_URL"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment = "o3-mini",
    api_version = "2024-12-01-preview"
)

## Configurate Funtion

In [2]:
from autogen_core.tools import FunctionTool
import requests

token = os.getenv("ADO_PERSONAL_ACCESS_TOKEN")

async def get_work_item_details(org: str, project: str, id: str) -> any:
    # Construct the URL for the API request
    url = f"https://dev.azure.com/{org}/{project}/_apis/wit/workitems/{id}?api-version=7.0"

    # using python's requests library to make the API call
    headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    response = requests.get(url, headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        work_item = response.json()
        return work_item
    else:
        # Handle errors (e.g., work item not found)
        print(f"Error: {response.status_code} - {response.text}")
        return None
get_work_item_details_tool = FunctionTool(get_work_item_details, description="Get one work item details from Azure DevOps.", name="get_work_item_details")

NameError: name 'os' is not defined

## Add memory

In [ ]:
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType

# Read from file doc/global-field-standards.md
with open("../doc/global-field-standards.md", "r") as f:
    global_field_standards = f.read()

# Initialize assistant memory with global field standards
assistant_memory = ListMemory()
await assistant_memory.add(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content=global_field_standards,
        metadata = {
            "name": "global-field-standards.md",
            "description": "Global field standards for Azure DevOps work items.",
        },
    )
)

In [ ]:
## Create Agent

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.ui import Console

agent = AssistantAgent(
    name = "assistant",
    system_message = "You are an experienced assistant with deep knowledge of Azure DevOps (ADO), particularly skilled in managing Work Items efficiently and accurately.",
    model_client = az_o3_model_client,
    tools = [get_work_item_details_tool],
    memory = [assistant_memory],
)
async def request_work_item_details():
    # Example request to get work item details
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Retrieve the details of work item '4850952' from the 'OneCRM' project in the 'dynamicscrm' organization, and verify whether all fields comply with the Global Field Standards. You should response `All fields are set` or tell me which required fields are not set", source="user")],
            CancellationToken(),
        ),
        output_stats=True,  # Enable stats printing.
    )

await request_work_item_details()